In [1]:

import time
import os
import json
from typing import Dict, Optional, Any
import sys
import re
import requests


project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [37]:
import os
#os.chdir('d:\github_practice\github_desktop\hands-on-time-series-analylsis-python')
print(os.getcwd())

d:\github_practice\github_desktop\AGI-Hackathon\RAG\legal_data


### 함수

In [3]:


def group_legal_texts(text_list):
    results = []
    # 1. 법률 이름 추출 (리스트의 첫 번째 요소)
    law_name = text_list[0].strip()
    
    # 2. index 1부터 시작해서, "제1조"로 시작하는 텍스트의 위치 찾기
    start_index = None
    for i in range(1, len(text_list)):
        if re.match(r'^제\d+조', text_list[i]):
            start_index = i
            break
    if start_index is None:
        # "제X조"가 하나도 없는 경우 빈 리스트 반환
        return results
    
    # 3. "제1조"부터 그룹핑 시작
    current_group = None
    # legal_prov_pattern 수정: "제숫자조" 이후 선택적으로 "의숫자"가 올 수 있고, 이후 열린 괄호부터 닫힌 괄호까지 모두 포함
    legal_prov_pattern = re.compile(r'^(제\d+조(?:의\d+)?\([^)]*\))')
    
    for text in text_list[start_index:]:
        text = text.strip()
        # 새 조의 시작인지 확인 (문자열이 "제"숫자"조" 로 시작)
        if re.match(r'^제\d+조', text):
            # 이미 진행 중인 조 그룹이 있다면 결과에 추가
            if current_group:
                results.append(current_group)
            # 새 조 그룹 시작: legal_provisions는 조 시작 부분의 풀네임
            m = legal_prov_pattern.match(text)
            if m:
                legal_provisions = m.group(1)
            else:
                # 패턴이 안 맞으면 첫 단어를 사용
                legal_provisions = text.split()[0]
            # 새 그룹 document 생성
            current_group = {
                "law_name": law_name,
                "legal_provisions": legal_provisions,
                "text": text  # 시작 텍스트 포함
            }
        else:
            # 새 조 시작이 아니면 현재 그룹에 이어 붙임
            if current_group:
                current_group["text"] += " " + text
    # 마지막 그룹도 결과에 추가
    if current_group:
        results.append(current_group)
    return results



In [43]:
def save_grouped_docs_as_json(grouped_docs):
    if not grouped_docs:
        print("저장할 데이터가 없습니다.")
        return
    
    # grouped_docs의 첫 번째 document에서 법률 이름 추출
    law_name = grouped_docs[0].get("law_name", "law").strip()
    
    # 저장할 디렉토리 경로 설정: legal_data_speak 폴더
    directory = "legal_data_insurance" # 법률 이름에 따라 하위 폴더 생성
    # 법률 이름에 따라 하위 폴더 생성 (예: legal_data_speak/개인정보보호법)
    os.makedirs(directory, exist_ok=True)
    
    # 파일 이름: 법률이름_docs.json (예: 개인정보보호법_docs.json)
    file_name = f"{law_name}_docs.json"
    file_path = os.path.join(directory, file_name)
    
    # JSON으로 저장 (ensure_ascii=False로 한글이 깨지지 않도록)
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(grouped_docs, f, ensure_ascii=False, indent=2)
    
    print(f"저장 완료: {file_path}")

In [5]:
def save_text_list_as_json(text_list):
    # 법률 이름 추출 (리스트의 첫 번째 요소)
    law_name = text_list[0].strip()
    # 저장할 디렉토리 경로 설정: legal_data_speak 폴더 (서브폴더 없이 파일 이름에 법률명을 포함)
    directory = "list_data"
    os.makedirs(directory, exist_ok=True)  # 디렉토리가 없으면 생성
    
    # 파일 이름: 법률이름_list.json (예: 개인정보보호법_list.json)
    file_name = f"{law_name}_list.json"
    file_path = os.path.join(directory, file_name)
    
    # JSON으로 저장 (ensure_ascii=False로 한글이 깨지지 않게 함)
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(text_list, f, ensure_ascii=False, indent=2)
    
    print(f"저장 완료: {file_path}")

### API Key

In [ ]:
# parsing api 사용
api_key = "your api key" # 여러분의 API Key를 입력하세요. 
url = "https://api.upstage.ai/v1/document-ai/document-parse" 
headers = { 
"Authorization": f"Bearer {api_key}" 
}

In [24]:
filename1 = "act_article_list_speak\개인정보 보호법(법률)(제19234호)(20250313).pdf"
filename2 = "act_article_list_speak\독점규제 및 공정거래에 관한 법률(법률)(제20712호)(20250121).pdf"
filename3 = "act_article_list_speak\소비자기본법(법률)(제20301호)(20250101).pdf"
filename4 = "act_article_list_speak\약관의 규제에 관한 법률(법률)(제20239호)(20240807).pdf"
filename5 = "act_article_list_speak\저작권법(법률)(제20358호)(20240828).pdf"
filename6 = "act_article_list_speak\전자금융거래법(법률)(제19734호)(20240915).pdf"
filename7 = "act_article_list_speak\전자상거래 등에서의 소비자보호에 관한 법률(법률)(제20302호)(20250214).pdf"
filename8 = "act_article_list_speak\정보통신망 이용촉진 및 정보보호 등에 관한 법률(법률)(제20260호)(20240814).pdf"

In [25]:
filename_h1 = "act_article_list_housing\상가건물 임대차보호법(법률)(제18675호)(20220104).pdf"
filename_h2 = "act_article_list_housing\주택임대차보호법(법률)(제19356호)(20230719).pdf"

In [64]:
filename_i1 = r"act_article_list_insurance\고용보험법(법률)(제20519호)(20250223).pdf"
filename_i2 = r"act_article_list_insurance\금융거래지표의 관리에 관한 법률(법률)(제16650호)(20201127).pdf"
filename_i3 = r"act_article_list_insurance\금융소비자 보호에 관한 법률(법률)(제20305호)(20240814).pdf"
filename_i4 = r"act_article_list_insurance\보험업법(법률)(제20436호)(20250131).pdf"
filename_i5 = r"act_article_list_insurance\1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법(법률)(제16029호)(20190625).pdf"

In [66]:
# filename #  번호만 바꿔서 아래 과정 시행 시 데이터 저장
files = {"document": open(filename_i5, "rb")} 
data = { 
"ocr": "True", # OCR을 강제로 수행하도록 설정 ("auto"로 설정 시 이미지 문서에서만 OCR 수행) 
"coordinates": False, # 각 레이아웃 요소의 위치 정보 반환 여부 
"chart_recognition": False, # 차트 인식 여부 (bar, line, pie 차트를 표로 변환) 
#"output_formats": "markdown", # 결과를 HTML 형식으로 반환 ("text", "markdown"도 가능) 
"base64_encoding": "['table']", # 표에 대한 base64 인코딩 요청 
"model": "document-parse" # 사용할 모델 지정 (가장 최신 모델 사용) 
} 
response = requests.post(url, headers=headers, files=files, data=data) 
print(response.json()) 

{'api': '2.0', 'content': {'html': '<header id=\'0\' style=\'font-size:14px\'>1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법</header>\n<h1 id=\'1\' style=\'font-size:22px\'>1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지<br>급에 관한 특별법 ( 약칭: 1959이전군퇴직금법 )</h1>\n<br><p id=\'2\' data-category=\'paragraph\' style=\'font-size:16px\'>[시행 2019. 6. 25.] [법률 제16029호, 2018. 12. 24., 일부개정]</p>\n<br><h1 id=\'3\' style=\'font-size:16px\'>국방부 (국방일자리정책과) 02-748-6638</h1>\n<p id=\'4\' data-category=\'paragraph\' style=\'font-size:16px\'>제1조(목적) 이 법은 현역에서 2년 이상 복무하고 1959년 12월 31일 이전에 이등상사 · 중사(1957년 1월 7일 이후의 계<br>급에 한한다) 또는 해군 일등병조 이상의 계급으로 퇴직한 군인에게 국가가 퇴직급여금을 지급하여 줌으로써 이들의<br>노후생활 안정을 도모하고 명예를 존중하며 국민의 애국정신함양에 이바지함을 목적으로 한다. <개정 2005. 3. 31.></p>\n<p id=\'5\' data-category=\'paragraph\' style=\'font-size:18px\'>제2조(정의) 이 법에서 사용하는 용어의 정의는 다음과 같다.</p>\n<p id=\'6\' data-category=\'list\' style=\'font-size:16px\'>1. "퇴직"이라 함은 전역 · 퇴역 · 제적의 경우를 말한다.<br>2. "퇴직급여금"이라 함은 1959년 12월 31일 이전에 퇴직함으로써 1960년 1월 1일 시행된 법률 제533호 공무원연<br>

### structured

In [17]:
full_content = response.json()['content']['html']
full_content 

'<p id=\'0\' data-category=\'paragraph\' style=\'font-size:20px\'>주택임대차보호법</p>\n<p id=\'1\' data-category=\'paragraph\' style=\'font-size:20px\'>주택임대차보호법 ( 약칭: 주택임대차법 )</p>\n<br><p id=\'2\' data-category=\'paragraph\' style=\'font-size:16px\'>[시행 2023. 7. 19.] [법률 제19356호, 2023. 4. 18., 일부개정]</p>\n<br><p id=\'3\' data-category=\'paragraph\' style=\'font-size:16px\'>법무부 (법무심의관실) 02-2110-3164<br>국토교통부 (주택임대차기획팀) 044-201-3348, 4178</p>\n<p id=\'4\' data-category=\'paragraph\' style=\'font-size:20px\'>제1조(목적) 이 법은 주거용 건물의 임대차(���)에 관하여 「민법」에 대한 특례를 규정함으로써 국민 주거생활의 안<br>정을 보장함을 목적으로 한다.</p>\n<br><p id=\'5\' data-category=\'paragraph\' style=\'font-size:16px\'>[전문개정 2008. 3. 21.]</p>\n<p id=\'6\' data-category=\'paragraph\' style=\'font-size:20px\'>제2조(적용 범위) 이 법은 주거용 건물(이하 "주택"이라 한다)의 전부 또는 일부의 임대차에 관하여 적용한다. 그 임차주<br>택(賃借住宅)의 일부가 주거 외의 목적으로 사용되는 경우에도 또한 같다.</p>\n<br><p id=\'7\' data-category=\'paragraph\' style=\'font-size:16px\'>[전문개정 2008. 3. 21.]</p>\n<p id=\'8\' data-category=\'paragra

In [18]:
! pip install openai

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
   ---------------------------------------- 0.0/599.1 kB ? eta -:--:--
   --------------------------------------- 599.1/599.1 kB 10.9 MB/s eta 0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)


In [ ]:
 
from openai import OpenAI  # openai==1.52.2
 # 여러분의 API Key를 입력하세요. 
 
client = OpenAI(
    api_key="your api key",
    base_url="https://api.upstage.ai/v1"
)
 
messages=[
    {
        "role": "system",
        "content": "You are an expert in information extraction. Extract information from the given HTML representation of image and organize them into a clear and accurate JSON format."
    },
    {
        "role": "user",
        "content" : full_content
    }
]
 
response_format={
    "type": "json_schema",
    "json_schema": {
        "name": "restaurant_receipt",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "documents": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                    "law_name": {
                        "type": "string",
                        "description": "법률의 이름"
                    },
                    "legal_provisions": {
                        "type": "string",
                        "description": "법률의 조항"
                    },
                    "text": {
                        "type": "string",
                        "description": "법령의 전체 내용"
                    }
                    },
                    "required": ["law_name", "legal_provisions", "text"],
                    "additionalProperties": False
                }
                }
            },
            "required": ["documents"],
            "additionalProperties": False
            }

    }
}
 
response = client.chat.completions.create(
    model="solar-pro",
    messages=messages,
    response_format=response_format
)
 


{ "documents": [ { "law_name": "주택임대차보호법", "legal_provisions": "제3조의7(임대인의 정보 제시 의무)", "text": "임대차계약을 체결할 때 임대인은 다음 각 호의 사항을 임차인에게 제시하여야 한다." }, { "law_name": "주택임대차보호법", "legal_provisions": "제5조 삭제", "text": "삭제된 조항" }, { "law_name": "주택임대차보호법", "legal_provisions": "제6조(계약의 갱신)", "text": "임대인이 임대차기간이 끝나기 6개월 전부터 2개월 전까지의 기간에 임차인에게 갱신거절의 통지를 하지 아니하거나 계약조건을 변경하지 아니한 경우, 그 기간이 끝난 때에 전 임대차와 동일한 조건으로 다시 임대차한 것으로 본다." }, { "law_name": "주택임대차보호법", "legal_provisions": "제6조의2(묵시적 갱신의 경우 계약의 해지)", "text": "임대차계약이 묵시적으로 갱신된 경우, 임차인은 언제든지 임대인에게 계약 해지를 통지할 수 있으며, 이 경우 해지는 임대인이 통지를 받은 날로부터 3개월 후에 효력이 발생한다." }, { "law_name": "주택임대차보호법", "legal_provisions": "제6조의3(계약갱신 요구 등)", "text": "임차인은 임대차기간이 끝나기 6개월 전부터 2개월 전까지의 기간에 임대인에게 계약 갱신을 요구할 수 있으며, 임대인은 정당한 사유 없이 이를 거절할 수 없다." } ] }


In [21]:
import json

# Parse the JSON content from the response
content_json = json.loads(response.choices[0].message.content)

# Print with pretty formatting
print(json.dumps(content_json, indent=2, ensure_ascii=False))

{
  "documents": [
    {
      "law_name": "주택임대차보호법",
      "legal_provisions": "제3조의7(임대인의 정보 제시 의무)",
      "text": "임대차계약을 체결할 때 임대인은 다음 각 호의 사항을 임차인에게 제시하여야 한다."
    },
    {
      "law_name": "주택임대차보호법",
      "legal_provisions": "제5조 삭제",
      "text": "삭제된 조항"
    },
    {
      "law_name": "주택임대차보호법",
      "legal_provisions": "제6조(계약의 갱신)",
      "text": "임대인이 임대차기간이 끝나기 6개월 전부터 2개월 전까지의 기간에 임차인에게 갱신거절의 통지를 하지 아니하거나 계약조건을 변경하지 아니한 경우, 그 기간이 끝난 때에 전 임대차와 동일한 조건으로 다시 임대차한 것으로 본다."
    },
    {
      "law_name": "주택임대차보호법",
      "legal_provisions": "제6조의2(묵시적 갱신의 경우 계약의 해지)",
      "text": "임대차계약이 묵시적으로 갱신된 경우, 임차인은 언제든지 임대인에게 계약 해지를 통지할 수 있으며, 이 경우 해지는 임대인이 통지를 받은 날로부터 3개월 후에 효력이 발생한다."
    },
    {
      "law_name": "주택임대차보호법",
      "legal_provisions": "제6조의3(계약갱신 요구 등)",
      "text": "임차인은 임대차기간이 끝나기 6개월 전부터 2개월 전까지의 기간에 임대인에게 계약 갱신을 요구할 수 있으며, 임대인은 정당한 사유 없이 이를 거절할 수 없다."
    }
  ]
}


### 시행

In [67]:
response.json()['elements']

[{'category': 'header',
  'content': {'html': "<header id='0' style='font-size:14px'>1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법</header>",
   'markdown': '',
   'text': ''},
  'id': 0,
  'page': 1},
 {'category': 'heading1',
  'content': {'html': "<h1 id='1' style='font-size:22px'>1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지<br>급에 관한 특별법 ( 약칭: 1959이전군퇴직금법 )</h1>",
   'markdown': '',
   'text': ''},
  'id': 1,
  'page': 1},
 {'category': 'paragraph',
  'content': {'html': "<br><p id='2' data-category='paragraph' style='font-size:16px'>[시행 2019. 6. 25.] [법률 제16029호, 2018. 12. 24., 일부개정]</p>",
   'markdown': '',
   'text': ''},
  'id': 2,
  'page': 1},
 {'category': 'heading1',
  'content': {'html': "<br><h1 id='3' style='font-size:16px'>국방부 (국방일자리정책과) 02-748-6638</h1>",
   'markdown': '',
   'text': ''},
  'id': 3,
  'page': 1},
 {'category': 'paragraph',
  'content': {'html': "<p id='4' data-category='paragraph' style='font-size:16px'>제1조(목적) 이 법은 현역에서 2년 이상 복무하고 1959년 12월 31일 이전에 이등상사 · 중사(1957년 1월

In [68]:
from bs4 import BeautifulSoup

# response.json()['elements']를 data 변수에 할당했다고 가정합니다.
data = response.json()['elements']

text_list = []

# 첫 번째 element: HTML 전체에서 텍스트 추출 (p 태그 여부 무관)
if data:
    first_item = data[0]
    html_content = first_item.get('content', {}).get('html', '')
    soup = BeautifulSoup(html_content, 'html.parser')
    first_text = soup.get_text(separator=' ', strip=True)
    text_list.append(first_text)

# 나머지 element들은 기존 방식으로 처리 (p 태그 내부의 텍스트만 추출)
for item in data[1:]:
    html_content = item.get('content', {}).get('html', '')
    soup = BeautifulSoup(html_content, 'html.parser')
    p_tag = soup.find('p')
    if p_tag:
        text = p_tag.get_text(separator=' ', strip=True)
        text_list.append(text)

text_list


['1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법',
 '[시행 2019. 6. 25.] [법률 제16029호, 2018. 12. 24., 일부개정]',
 '제1조(목적) 이 법은 현역에서 2년 이상 복무하고 1959년 12월 31일 이전에 이등상사 · 중사(1957년 1월 7일 이후의 계 급에 한한다) 또는 해군 일등병조 이상의 계급으로 퇴직한 군인에게 국가가 퇴직급여금을 지급하여 줌으로써 이들의 노후생활 안정을 도모하고 명예를 존중하며 국민의 애국정신함양에 이바지함을 목적으로 한다. <개정 2005. 3. 31.>',
 '제2조(정의) 이 법에서 사용하는 용어의 정의는 다음과 같다.',
 '1. "퇴직"이라 함은 전역 · 퇴역 · 제적의 경우를 말한다. 2. "퇴직급여금"이라 함은 1959년 12월 31일 이전에 퇴직함으로써 1960년 1월 1일 시행된 법률 제533호 공무원연 금법에 의한 퇴직연금 및 퇴직일시금의 혜택을 받지 못한 군인에게 지급하는 퇴직일시금에 준하는 보상성격의 급여금을 말한다. 3. "유족"이라 함은 다음 각목의 1에 해당하는 자로서 퇴직자의 민법상 재산상속인을 말한다. 가. 배우자(사실상 혼인관계에 있던 자를 포함한다) 나. 자녀 다. 부모 라. 손자녀',
 '제3조(적용대상자) 이 법은 1948년 8월 15일 이후부터 1959년 12월 31일 사이에 현역에서 2년 이상 복무하고 퇴직 당 시 이등상사 · 중사(1957년 1월 7일 이후의 계급에 한한다) 또는 해군 일등병조 이상의 계급에 있었던 자로서 1959년 12월 31일 이전에 퇴직함으로써 국가로부터 퇴직연금 및 퇴직일시금을 지급받지 못한자 및 그 유족에 대하 여 적용한다. 다만, 「공무원연금법」 · 「군인연금법」 및 「사립학교교직원 연금법」 등에 의하여 1959년 12월 31일 이 전의 군복무기간을 합산받아 퇴직연금 및 퇴직일시금을 지급받았거나 지급받고 있는 자 및 그 유족은 그러하지 아 니하다. <개정 2005. 3. 31.>',
 '제4조(퇴직급여

In [69]:
save_text_list_as_json(text_list)

저장 완료: list_data\1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법_list.json


In [70]:
grouped_docs = group_legal_texts(text_list)
print(json.dumps(grouped_docs, ensure_ascii=False, indent=2))

[
  {
    "law_name": "1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법",
    "legal_provisions": "제1조(목적)",
    "text": "제1조(목적) 이 법은 현역에서 2년 이상 복무하고 1959년 12월 31일 이전에 이등상사 · 중사(1957년 1월 7일 이후의 계 급에 한한다) 또는 해군 일등병조 이상의 계급으로 퇴직한 군인에게 국가가 퇴직급여금을 지급하여 줌으로써 이들의 노후생활 안정을 도모하고 명예를 존중하며 국민의 애국정신함양에 이바지함을 목적으로 한다. <개정 2005. 3. 31.>"
  },
  {
    "law_name": "1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법",
    "legal_provisions": "제2조(정의)",
    "text": "제2조(정의) 이 법에서 사용하는 용어의 정의는 다음과 같다. 1. \"퇴직\"이라 함은 전역 · 퇴역 · 제적의 경우를 말한다. 2. \"퇴직급여금\"이라 함은 1959년 12월 31일 이전에 퇴직함으로써 1960년 1월 1일 시행된 법률 제533호 공무원연 금법에 의한 퇴직연금 및 퇴직일시금의 혜택을 받지 못한 군인에게 지급하는 퇴직일시금에 준하는 보상성격의 급여금을 말한다. 3. \"유족\"이라 함은 다음 각목의 1에 해당하는 자로서 퇴직자의 민법상 재산상속인을 말한다. 가. 배우자(사실상 혼인관계에 있던 자를 포함한다) 나. 자녀 다. 부모 라. 손자녀"
  },
  {
    "law_name": "1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법",
    "legal_provisions": "제3조(적용대상자)",
    "text": "제3조(적용대상자) 이 법은 1948년 8월 15일 이후부터 1959년 12월 31일 사이에 현역에서 2년 이상 복무하고 퇴직 당 시 이등상사 · 중사(1957년 1월 7일 이후의 계급에 한한다) 또는 해군 일등병조 이

In [71]:
save_grouped_docs_as_json(grouped_docs)

저장 완료: legal_data_insurance\1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법_docs.json
